In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
print("Imported Successfully")

# Function to load and preprocess images for training or validation
def load_and_preprocess_images(folder_path, is_validation=False):
    images, labels = [], []
    label_keyword = "positive" if is_validation else ""
    for image_file in tqdm(os.listdir(folder_path), desc=f"Loading {folder_path}"):
        image = cv2.imread(os.path.join(folder_path, image_file))
        image = cv2.resize(image, (64, 64))
        image = np.stack([image, image, image], axis=2)
        images.append(image)
        if is_validation:
            labels.append(True if "positive" in image_file else False)
        else:
            labels.append(int(image_file.split('_')[1].split('.')[0]))
    return np.array(images, dtype=np.float32) / 255.0, np.array(labels)

# Load train and validation data
train_images, train_labels = load_and_preprocess_images('/kaggle/input/test-train-data/test_train_data')
val_images, val_labels = load_and_preprocess_images('/kaggle/input/p-val-data/processed_validation_data', is_validation=True)

# Preprocess the image data
print("Pre-Processing...")
train_images = np.array(train_images, dtype=np.float32) / 255.0
val_images = np.array(val_images, dtype=np.float32) / 255.0
train_labels, val_labels = np.array(train_labels), np.array(val_labels)
num_classes = 60000

# Build the CNN model
print("Building Model...")
model = Sequential()
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
model.add(resnet50)
model.add(Flatten())
model.add(Dense(num_classes, activation='sigmoid'))
print("Compiling in Progress...")
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model, "\nModel Built & Compiled")

Imported Successfully


In [ ]:
# Train the model
print("Training in Progress...")
with tqdm(total=100) as pbar:
  for epoch in range(100):
    model.fit(train_images, train_labels, epochs=1, validation_data=(val_images, val_labels))
    pbar.update(1)
    
# Save the trained model
model.save('trained_model.h5')
print("Trained Model Saved!")